# Set up `Observation` objects for each Exfor subentry

Internally, this involves initializing a `jitr.xs.elastic.DifferentialWorkspace` for each `Observation`, which precomputes things like boundary conditions to speed up computation of observables for an interaction parameter.

In [1]:
import numpy as np

In [2]:
import rxmc

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


In [3]:
import lzma
import pickle
from pathlib import Path

from tqdm import tqdm

output_dir = Path("./data/")

In [4]:
with lzma.open(output_dir / "pp_elastic_data.xz", "rb") as f:
    pp_elastic_measurements = pickle.load(f)
with lzma.open(output_dir / "nn_elastic_data.xz", "rb") as f:
    nn_elastic_measurements = pickle.load(f)

In [ ]:
nn_observations = {}
for entry_id, mlist in tqdm(nn_elastic_measurements.items()):
    if entry_id not in nn_observations:
        nn_observations[entry_id] = []
    for rxn, m in mlist:
        nn_observations[entry_id].append(
            rxmc.elastic_diffxs_observation.ElasticDifferentialXSObservation(
                measurement=m,
                reaction=rxn,
                quantity="dXS/dA",
                lmax=50,
                angles_vis=np.linspace(
                    max(0, np.min(m.x) * 0.9), min(180, np.max(m.x) * 1.1), 100
                ),
                ObservationClass=rxmc.observation.Observation,
                error_kwargs={
                    "include_sys_norm_err": False,
                    "include_sys_offset_err": False,
                },
            )
        )

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
pp_observations = {}
for entry_id, mlist in tqdm(pp_elastic_measurements.items()):
    if entry_id not in pp_observations:
        pp_observations[entry_id] = []
    for rxn, m in mlist:
        pp_observations[entry_id].append(
            rxmc.elastic_diffxs_observation.ElasticDifferentialXSObservation(
                m,
                rxn,
                "dXS/dRuth",
                50,
                angles_vis=np.linspace(
                    max(0, np.min(m.x) * 0.9), min(180, np.max(m.x) * 1.1), 100
                ),
                ObservationClass=rxmc.observation.Observation,
                error_kwargs={
                    "include_sys_norm_err": False,
                    "include_sys_offset_err": False,
                },
            )
        )

In [ ]:
with lzma.open(output_dir / "pp_elastic_observations.xz", "wb") as f:
    pickle.dump(pp_observations, f)
with lzma.open(output_dir / "nn_elastic_observations.xz", "wb") as f:
    pickle.dump(nn_observations, f)

## Some tests

In [ ]:
with open("../likelihood_model/likelihood_model.pkl", "rb") as f:
    lm = pickle.load(f)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
pp_observations.keys()

In [ ]:
obs = pp_observations["O0387"][0]

In [ ]:
y = 0.9 * obs.y + 0.01

In [ ]:
plt.errorbar(
    obs.x,
    obs.y,
    np.diag(np.sqrt(obs._obs.statistical_covariance)),
    linestyle="none",
)
plt.plot(obs.x, y)
# plt.yscale("log")

In [ ]:
np.sum((y - obs.y) ** 2 / ((y * 0.3) ** 2 + np.diag(obs._obs.statistical_covariance)))

In [ ]:
lm.chi2(obs, y)